In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import yaml
import re

import seaborn as sns
import numpy as np
import pandas as pd

from functools import reduce

from fiber.cohort import Cohort
from fiber.condition import Procedure, Diagnosis, Drug, VitalSign, Patient
from fiber.database.hana import engine, Session, print_sqla

from fiber.database.table import fact

# Cohort Definition

In [ ]:
min_age = Patient.age_in_days > 365 * 18
heart_surgery_condition = Procedure(code='35.%').with_(min_age) | Procedure(code='36.1%').with_(min_age)

In [ ]:
heart_surgery_cohort = Cohort(heart_surgery_condition)

In [ ]:
len(heart_surgery_cohort)

# Demographics

In [ ]:
demographics = heart_surgery_cohort.demographics

In [ ]:
demographics["age"]["figure"]

In [ ]:
demographics["gender"]["figure"]

# Comorbidities

In [ ]:
aki = heart_surgery_cohort.has_onset(
    name="aki",
    condition=Diagnosis(code="584.9", context="ICD-9"), 
    time_deltas=[1, 7, 14, 28]
)
aki.head()

In [ ]:
stroke = heart_surgery_cohort.has_onset(
    name="stroke",
    condition=Diagnosis(code='433.%') | Diagnosis(code='434.%') | Diagnosis(code='436.%'), 
    time_deltas=[1, 7, 14, 28]
)
stroke.head()

# Preconditions

In [ ]:
renal_failure_condition = Diagnosis.from_condition_store(name="renal failure")

renal_failure = heart_surgery_cohort.has_precondition(
    name="renal_failure",
    condition=renal_failure_condition
)
renal_failure.head()

In [ ]:
cardiac_arrhythmias = heart_surgery_cohort.has_precondition(
    name="cardiac_arrhythmias",
    condition=Diagnosis.from_condition_store(name="cardiac arrhythmias")
)

# One dataframe to rule them all

In [ ]:
medical_record_numbers = pd.DataFrame(heart_surgery_cohort, columns=["medical_record_number"])

In [ ]:
patient_data = heart_surgery_cohort.get(Patient())

In [ ]:
df = pd.concat([
    medical_record_numbers,
    patient_data.drop(columns="medical_record_number"),
    cardiac_arrhythmias.cardiac_arrhythmias,
    renal_failure.renal_failure,
    stroke.drop(columns="medical_record_number"),
    aki.drop(columns="medical_record_number")
], axis=1)

In [ ]:
df.tail(10)

### Experimental

In [ ]:
crea = heart_surgery_cohort.lab_results_for('CREATININE-SERUM')

In [ ]:
crea['develops_aki'] = crea.MEDICAL_RECORD_NUMBER.isin(df[df.aki_28_days].medical_record_number)

In [ ]:
crea = crea[crea.VALUE < 10]

_, bins = np.histogram(crea['VALUE'])
g = sns.FacetGrid(crea, hue="develops_aki", height=4, aspect=1)
g = g.map(sns.distplot, "VALUE", bins=bins, hist=False)